# Compression Analysis

Andrei's analysis on the impact of various compression parameters on file size, compression time, and PSNR.

If time allows it, I will also look into decompression time but it's not high priority for now.

## Dataset

I sampled 2,500 images from the pre-release Pit30M data, compressed **losslessly** with WebP. The images are sampled at random from 25 different driving logs and 7 cameras, covering urban areas, parks, traffic, etc., as well as a variety of weather and driving condition.

The average image size is 2183KiB, computed with this handy one-liner:

```bash
find $DATASET_DIR -iname '*.webp' -exec ls -la '{}' + | \
    awk '{ total_b+=$5; count+=1 } END { print total_b/count/1024 }' 
```

Given that this covers people*, cars*, trees, buildings, roads, etc. as seen from different view points and focal length, it should give us pretty good signal.

*) In the final dataset release, faces and license plates were blurred to preserve privacy.

In [6]:
import io
import os
import random
import sys
import time
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import skimage.metrics as skm
from joblib import Parallel, delayed, Memory
from tqdm import tqdm
from PIL import Image

%matplotlib inline

memory = Memory(location="/tmp/compression-experiment-pit30m")

print(sys.version_info)
print(sys.executable)

sys.version_info(major=3, minor=9, micro=0, releaselevel='final', serial=0)
/home/andrei/miniconda3/envs/devkit309-dev/bin/python


In [7]:
DATASET_DIR = Path("~/work/pit30m/out/size_analysis").expanduser()
# Limit the number of images to make the computation complete on time
#
# Especially SSIM computation is slow with the skimage implementation
MAX_IMG = 2500

random.seed(123)
image_fpaths = sorted(list(DATASET_DIR.glob("**/*.webp")))
random.shuffle(image_fpaths)
image_fpaths = image_fpaths[:MAX_IMG]
print(len(image_fpaths))

2500


In [8]:
# all_images = [(fpath, Image.open(fpath)) for fpath in image_fpaths]

In [ ]:

# Default in scikit-image and in libvmaf, which means it's likely a reasonable default.
# ffmpeg uses 8x8 windows, so close to this too. See: https://arxiv.org/pdf/2101.06354.pdf
ssim_win_size = 11

@memory.cache()
def run_configuration(all_images, format, quality, method):
    # NOTE: For webm 'method = 4' is the default, with lower being faster

    fpaths = []
    encode_times = []
    decode_times = []
    psnrs = []
    ssims = []
    sizes_b = []

    for fpath in tqdm(all_images, postfix=f"{format} q={quality} m={method}"):
        image = Image.open(fpath)
        
        with io.BytesIO() as f:
            start_ts = time.time()
            if format == "webp":
                image.save(f, format=format, quality=quality, method=method)
            else:
                # JPEG does not have 'method' to tweak encoding speed.
                image.save(f, format=format, quality=quality)
            encode_time = time.time() - start_ts
            encode_times.append(encode_time)
            n_bytes = len(f.getbuffer())

            f.seek(0)
            decode_start_ts = time.time()
            reloaded_compressed_np = np.asarray(Image.open(f))
            decode_time = time.time() - decode_start_ts
            decode_times.append(decode_time)
            original_np = np.asarray(image)
            psnr_db = skm.peak_signal_noise_ratio(original_np, reloaded_compressed_np)
#             print(original_np.shape)
#             print(reloaded_compressed_np.shape)
            # NOTE(andrei): SSIM right now is a bit slow...
            ssim = skm.structural_similarity(original_np, reloaded_compressed_np, 
                                             win_size=ssim_win_size, channel_axis=2)
#             ssim = -1

            fpaths.append(fpath)
            sizes_b.append(n_bytes)
            psnrs.append(psnr_db)
            ssims.append(ssim)

#             plt.figure(figsize=(20, 8))
#             plt.imshow(reloaded_compressed_np)

    return fpaths, encode_times, decode_times, psnrs, ssims, sizes_b

configs = []
for format in ["jpeg", "webp"]:
    for quality in [65, 70, 75, 80, 85, 90, 95, 100]:
        meths = [-1] if format == "jpeg" else [0, 1, 2, 3, 4, 5, 6]
        for method in meths:
            configs.append((format, quality, method))
 

# Run the main experiment multithreaded, then do the timing analysis separately, or reload the
# older experiment you ran single-threaded. We mostly care about the SSIM for this parallelized run,
# since SSIM is slow AF to compute.
pool = Parallel(n_jobs=8)
results = pool(delayed(run_configuration)(image_fpaths, format, quality, method)
               for format, quality, method in configs)
# results[(format, quality, method)] = \
#                 run_configuration(image_fpaths, format=format,
#                                   quality=quality, method=method)

________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-3732551948.run_configuration...
run_configuration([ PosixPath('/home/andrei/work/pit30m/out/size_analysis/f81c4def-1e47-4b83-cff8-3a8d2f9e5538/hdcam_10_port_front_roof_wide/0013469.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/4644d0d4-c4c9-4dad-dbaf-b66c4932178c/hdcam_12_middle_front_roof_wide/0003797.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/46d5d171-8471-441a-e67e-3d434949f8ea/hdcam_12_middle_front_roof_narrow_right/0022236.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/9dd6c8d1-1461-459c-e2b8-92682ab1807e/hdcam_08_port_rear_roof_wide/0010315.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/4644d0d4-c4c9-4dad-dbaf-b66c4932178c/hdcam_12_middle_front_roof_wide/0000167...., 
'jpeg', 65, -1)
________________________________________________________________________________
[Memory] Calling __main__

 99%|█████████▉| 2479/2500 [1:03:22<00:31,  1.51s/it, jpeg q=85 m=-1]]

____________________________________________run_configuration - 3801.9s, 63.4min
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-3732551948.run_configuration...
run_configuration([ PosixPath('/home/andrei/work/pit30m/out/size_analysis/f81c4def-1e47-4b83-cff8-3a8d2f9e5538/hdcam_10_port_front_roof_wide/0013469.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/4644d0d4-c4c9-4dad-dbaf-b66c4932178c/hdcam_12_middle_front_roof_wide/0003797.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/46d5d171-8471-441a-e67e-3d434949f8ea/hdcam_12_middle_front_roof_narrow_right/0022236.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/9dd6c8d1-1461-459c-e2b8-92682ab1807e/hdcam_08_port_rear_roof_wide/0010315.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/4644d0d4-c4c9-4dad-dbaf-b66c4932178c/hdcam_12_middle_front_roof_wide/0000167...., 
'webp', 65, 0)


100%|█████████▉| 2491/2500 [1:03:22<00:12,  1.44s/it, jpeg q=80 m=-1]

____________________________________________run_configuration - 3802.8s, 63.4min
________________________________________________________________________________
[Memory] Calling __main__--tmp-ipykernel-3732551948.run_configuration...
run_configuration([ PosixPath('/home/andrei/work/pit30m/out/size_analysis/f81c4def-1e47-4b83-cff8-3a8d2f9e5538/hdcam_10_port_front_roof_wide/0013469.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/4644d0d4-c4c9-4dad-dbaf-b66c4932178c/hdcam_12_middle_front_roof_wide/0003797.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/46d5d171-8471-441a-e67e-3d434949f8ea/hdcam_12_middle_front_roof_narrow_right/0022236.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/9dd6c8d1-1461-459c-e2b8-92682ab1807e/hdcam_08_port_rear_roof_wide/0010315.webp'),
  PosixPath('/home/andrei/work/pit30m/out/size_analysis/4644d0d4-c4c9-4dad-dbaf-b66c4932178c/hdcam_12_middle_front_roof_wide/0000167...., 
'webp', 65, 1)


...

 77%|███████▋  | 1925/2500 [52:28<08:52,  1.08it/s, webp q=100 m=6]

____________________________________________run_configuration - 3433.3s, 57.2min


 83%|████████▎ | 2076/2500 [54:50<06:52,  1.03it/s, webp q=100 m=6]

____________________________________________run_configuration - 3375.6s, 56.3min


 93%|█████████▎| 2321/2500 [58:37<02:49,  1.06it/s, webp q=100 m=6]

In [ ]:
import pickle as pkl

with open("/home/andrei/work/pit30m/compression-blog-results-4k-2022-11-08.pkl", "rb") as f:
    # Get the timing info from here
    results_11_08 = pkl.load(f)

In [ ]:
# Let's plot PSNR vs. quality (pretty predictible)
# 
# Box plots would be interesting in case there's some configs which cause unexpected spread
%matplotlib notebook
from collections import defaultdict

plot = defaultdict(lambda: defaultdict(dict))
size_plot = defaultdict(lambda: defaultdict(dict))
compression_speed_plot = defaultdict(lambda: defaultdict(dict))
decompression_speed_plot = defaultdict(lambda: defaultdict(dict))

# Buffer for timing metrics, ignore the first and last 'buffer' measurements
buffer = 3

# Track all fpaths even if redundant to make it easier to datamine and grab specific images, e.g.
# high filesize but low PSNR images (hard to compress).
all_fpaths = []
all_psnrs = []
all_sizes_b = []
all_qs = []
all_methods = []

for (fmt, quality, method), (fpaths, enc_t, dec_t, psnrs, ssims, sizes_b) in results.items():
    assert quality not in plot
    enc_t_b = enc_t[buffer:-buffer]
    dec_t_b = dec_t[buffer:-buffer]
    decompression_speed_plot[fmt][method][quality] = np.mean(dec_t_b), np.std(dec_t_b)
    compression_speed_plot[fmt][method][quality] = np.mean(enc_t_b), np.std(enc_t_b)
    
    plot[fmt][method][quality] = np.mean(psnrs), np.std(psnrs)
    size_plot[fmt][method][quality] = np.mean(sizes_b), np.std(sizes_b)

    all_fpaths += fpaths
    all_sizes_b += sizes_b
    all_psnrs += psnrs
    all_qs += [quality] * len(psnrs)
    all_methods += [method] * len(psnrs)
    
    
all_sizes_b = np.array(all_sizes_b)
all_psnrs = np.array(all_psnrs)
all_qs = np.array(all_qs)
all_methods = np.array(all_methods)
    
plt.figure()
for fmt in plot:
    for method in plot[fmt]:
        xx, yy, yerr = [], [], []
        for q, (mean_psnr, std) in plot[fmt][method].items():
            xx.append(q)
            yy.append(mean_psnr)
            yerr.append(std)

        label = f"Method: {method} @ {fmt}"
        m = '' if fmt == "webp" else "^"
        if method == 2:
            # For debug
            m = 'o'
    #     plt.errorbar(xx, yy, yerr=yerr, capsize=10, label=label)
        plt.plot(xx, yy, label=label, marker=m)
       


plt.xlabel("WebP quality parameter")
plt.ylabel("Resulting mean PSNR over dataset")
# plt.ylim(30, 50)
plt.legend()
plt.grid("on")

plt.figure()
for fmt in size_plot:
    for method in size_plot[fmt]:
        xx, yy, yerr = [], [], []
        for q, (mean_sz, std) in size_plot[fmt][method].items():
            xx.append(q)
            yy.append(mean_sz/1024)
            yerr.append(std/1024)
        label = f"Format: {fmt}, Method: {method}"
        m = '' if fmt == "webp" else "^"
    #     plt.errorbar(xx, yy, yerr=yerr, capsize=10, label=label)
        plt.plot(xx, yy, label=label, marker=m)
        
plt.xlabel("WebP quality parameter")
plt.ylabel("File size, KiB")
plt.ylim(0, 1024)
plt.legend()
plt.grid("on")
plt.tight_layout()


plt.figure()
for fmt in compression_speed_plot:
    for method in compression_speed_plot[fmt]:
        xx, yy, yerr = [], [], []
        for q, (mean_sz, std) in compression_speed_plot[fmt][method].items():
            xx.append(q)
            yy.append(mean_sz * 1000)
            yerr.append(std * 1000)
        label = f"Method: {method}, fmt = {fmt}"
        m = '' if fmt == "webp" else "^"
#         plt.errorbar(xx, yy, yerr=yerr, capsize=10, label=label, marker=m)
        plt.plot(xx, yy, label=label, marker=m)
        
plt.xlabel("WebP quality parameter")
plt.ylabel("Compression speed (ms)")

plt.ylim(0, 300)
plt.legend()
plt.grid("on")
plt.tight_layout()

#
# Space as a function of method (WebP-specific)
#
plt.figure()
for compression_level in size_plot["webp"][3]:
    for fmt in ["webp"]:
        xx, yy, yerr = [], [], []
        for method in size_plot[fmt]:
            mean_sz, std = size_plot[fmt][method][compression_level]
            xx.append(method)
            yy.append(mean_sz / 1024)
            yerr.append(std / 1024)

        label = f"Format: {fmt}, Quality: {compression_level}"
        plt.errorbar(xx, yy, yerr=yerr, capsize=10, label=label)

# TODO(andrei): Roughly map method to time factor (absolute difference depends on quality)
plt.xlabel("Method (i.e. computational budget)")
plt.ylabel("Mean size (KiB)")
plt.legend()
plt.grid("on")
plt.tight_layout()


In [ ]:

    
plt.figure()
for fmt in decompression_speed_plot:
    for method in decompression_speed_plot[fmt]:
        if fmt == "jpeg" and method != -1:
            continue
            
        xx, yy, yerr = [], [], []
        for q, (mean_sz, std) in decompression_speed_plot[fmt][method].items():
            xx.append(q)
            yy.append(mean_sz * 1000)
            yerr.append(std * 1000)
        label = f"Method: {fmt} / {method}"
#         plt.errorbar(xx, yy, yerr=yerr, capsize=10, label=label)
        m = '' if fmt == "webp" else "^"
        plt.plot(xx, yy, label=label, marker=m)
        
plt.xlabel("WebP / JPEG quality parameter")
plt.ylabel("Decompression time (ms)")

plt.ylim(0, 50)
plt.legend()
plt.grid("on")
plt.tight_layout()

In [ ]:
# Mean size for each PSNR bucket
psnr_bins = np.linspace(32, 45, num=10)

for q in [100, 95, 90, 85, 80]:
    foo = all_psnrs[(all_qs == q) & (all_methods == 3)]
    bar = all_sizes_b[(all_qs == q) & (all_methods == 3)]

    idxs = np.digitize(foo, psnr_bins)

    res = []
    res_std = []
    for bidx, bin in enumerate(psnr_bins):
        res.append(np.mean(bar[idxs == bidx]) / 1024)
        res_std.append(np.std(bar[idxs == bidx]) / 1024)

    plt.errorbar(psnr_bins, res, yerr=res_std, capsize=8, label=f"{q = }")
    
plt.legend()
plt.xlabel("PSNR")
plt.ylabel("Size (KiB)")
plt.ylim(0, 1024)

In [ ]:
#
# PSNR vs. File Size
#
print(len(all_sizes_b))
print(len(all_psnrs))
c = all_qs
ax = plt.scatter(all_psnrs, np.array(all_sizes_b) / 1024, c=c, s=0.1) 
plt.colorbar(ax)
plt.ylim(0, 1024)
plt.xlabel("PSNR")
plt.ylabel("File Size (KiB)")
plt.title("Colored by WebP quality")

### Some conclusion
 - initial storage analysis used WebP method 4 at q=85
 - using JPEG at q85 would increase image space by 50% so we probably can't afford it :(
    - current estimate sits at roughly 160TiB conservatively.
    - images become ~50% of the storage if q=85 webp
    - if we increase image size by 50%, increase total storage by 25%, so ~200TiB, still within the 300TiB
    - jpeg at 90 means ~400k images, 100% increase in images, 50% total, so 240TiB, may be a little too much

 - for webp max compression use at least method = 2
 - for webp, method = 2 is an outlier in terms of PSNR... either use method = 3 or 0/1
     - of course 0/1 will produce larger files, but not by a lot 
     - prioritize PSNR over very tiny files!
 - method 3 and 4 are identical in compression speed!
 - method 2 is way faster than 3, ~2.3x faster
 - method 1 is is 3x faster than 3 ---> we should use '1'
     - storage of method 1 vs. JPEG: 307 --> 250kb, though JPEG PSNR at q=85 is higher
     
     
=> just do method=1 webp at q=90, instead of 2/4 at q=85.

# Detailed PSNR analysis

Maybe look at CDFs? Find outliers where there are huge PSNR gaps between methods?

In [ ]:
# Find outliers
high_file_low_psnr = (all_qs == 85) & (all_methods == 3) & (all_sizes_b > 600) & (all_psnrs < 40)
high_file_low_psnr.sum(), "matches"